# Titanic submission

In [45]:
from pandas.core.common import SettingWithCopyWarning
import warnings
from tensorflow import keras
import utils
import pandas as pd

warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [46]:
from loadDataUtils import loadDataUtils

In [47]:
path_train = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\train.csv'
path_test = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\test.csv'
data = loadDataUtils(path_train, path_test)

In [48]:
df_train, df_test = data.get_train_and_test()

### Clean data

In [49]:
from titanicPreprocessing import preprocess

In [50]:
p = preprocess(df_train.copy(), df_test.copy())
p.do_preprocess()

In [51]:
train, test = p.get_data()

In [52]:
train_target = train['Survived']
train.drop(columns=['Survived'], inplace=True)

In [53]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        889 non-null    int64  
 1   Pclass             889 non-null    int64  
 2   Age                889 non-null    float64
 3   SibSp              889 non-null    int64  
 4   Parch              889 non-null    int64  
 5   Fare               889 non-null    float64
 6   cabin_multiple     889 non-null    int64  
 7   Sex_female         889 non-null    uint8  
 8   Sex_male           889 non-null    uint8  
 9   Embarked_C         889 non-null    uint8  
 10  Embarked_Q         889 non-null    uint8  
 11  Embarked_S         889 non-null    uint8  
 12  cabin_letter_0     889 non-null    uint8  
 13  cabin_letter_A     889 non-null    uint8  
 14  cabin_letter_B     889 non-null    uint8  
 15  cabin_letter_C     889 non-null    uint8  
 16  cabin_letter_D     889 non

In [54]:
test.insert(20, 'cabin_letter_T', 0)

In [55]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        418 non-null    int64  
 1   Pclass             418 non-null    int64  
 2   Age                418 non-null    float64
 3   SibSp              418 non-null    int64  
 4   Parch              418 non-null    int64  
 5   Fare               418 non-null    float64
 6   cabin_multiple     418 non-null    int64  
 7   Sex_female         418 non-null    uint8  
 8   Sex_male           418 non-null    uint8  
 9   Embarked_C         418 non-null    uint8  
 10  Embarked_Q         418 non-null    uint8  
 11  Embarked_S         418 non-null    uint8  
 12  cabin_letter_0     418 non-null    uint8  
 13  cabin_letter_A     418 non-null    uint8  
 14  cabin_letter_B     418 non-null    uint8  
 15  cabin_letter_C     418 non-null    uint8  
 16  cabin_letter_D     418 non

## Load pickle

In [34]:
pickle = utils.load_pickle_obj('./save_best_model/pickle_best_models')

In [35]:
svc_best = pickle[0]
xgb_best = pickle[1]
voting_clf_best = pickle[2]

model = keras.models.load_model("./save_best_model_pickle/keras_classifier3")

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                448       
                                                                 
 dense_5 (Dense)             (None, 6)                 102       
                                                                 
 dense_6 (Dense)             (None, 1)                 7         
                                                                 
Total params: 557
Trainable params: 557
Non-trainable params: 0
_________________________________________________________________


### fit on all train data

In [37]:
svc_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)
xgb_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)
voting_clf_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)
model.fit(train.loc[:, train.columns != 'PassengerId'], train_target, epochs =25)

[14:11:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytre" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Epoch 1/25
28/28 [==============================] - 0s 1ms/step - loss: 0.4101 - accuracy: 0.8268
Epoch 2/25
28/28 [==============================] - 0s 1ms/step - loss: 0.4088 - accuracy: 0.8313
Epoch 3/25
28/28 [==============================] - 0s 2ms/step - loss: 0.4077 - accuracy: 0.8313
Epoch 4/25
28/28 [==============================] - 0s 1ms/step - loss: 0.4072 - accuracy: 0.8301
Epoch 5/25
28/28 [==============================] - 0s 582us/step - loss: 0.4063 - accuracy: 0.8335
Epoch 6/25
28/28 [==============================] - 0s 1ms/step - loss: 0.4061 - acc

In [38]:
import numpy as np

y_hat_svc_best = svc_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)
y_hat_xgb_best = xgb_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)
y_hat_voting_clf_best = voting_clf_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)
y_hat_model = np.round(model.predict(test.loc[:, test.columns != 'PassengerId'])).astype(int)

In [39]:
y_hat_model = y_hat_model[:,0]
y_hat_model.shape

(418,)

## Convert results to dataframe

In [40]:
final_data = {'PassengerId': test.PassengerId, 'Survived': y_hat_svc_best}
submission = pd.DataFrame(data=final_data)

final_data_2 = {'PassengerId': test.PassengerId, 'Survived': y_hat_xgb_best}
submission_2 = pd.DataFrame(data=final_data_2)

final_data_3 = {'PassengerId': test.PassengerId, 'Survived': y_hat_voting_clf_best}
submission_3 = pd.DataFrame(data=final_data_3)

final_data_4 = {'PassengerId': test.PassengerId, 'Survived': y_hat_model}
submission_4 = pd.DataFrame(data=final_data_4)

In [41]:
final_data_comp = {'PassengerId': test.PassengerId, 'Survived_svc': y_hat_svc_best, 'Survived_xgb': y_hat_xgb_best, 'Survived_voting' : y_hat_voting_clf_best, 'Survived_model' : y_hat_model}
comparison = pd.DataFrame(data=final_data_comp)

In [42]:
#track differences between outputs
comparison['difference_svc_xgb'] = comparison.apply(lambda x: 1 if x.Survived_svc != x.Survived_xgb else 0, axis =1)
comparison['difference_svc_voting'] = comparison.apply(lambda x: 1 if x.Survived_svc != x.Survived_voting else 0, axis =1)
comparison['difference_svc_model'] = comparison.apply(lambda x: 1 if x.Survived_model != x.Survived_svc else 0, axis =1)

In [43]:
comparison.difference_svc_model.value_counts()

0    380
1     38
Name: difference_svc_model, dtype: int64

## Prepare to submit

In [44]:
#prepare submission files
submission.to_csv('./submission/submission_svc.csv', index =False)
submission_2.to_csv('./submission/submission_xgb.csv',index=False)
submission_3.to_csv('./submission/submission_voting.csv', index=False)
submission_4.to_csv('./submission/submission_model.csv', index=False)